In [66]:
import pandas as pd
import numpy as np
import copy, time
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

In [67]:
data_path = 'data/'
df_train = pd.read_csv(data_path +'house_train.csv.gz')
df_test = pd.read_csv(data_path +'house_test.csv.gz')

train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'], axis = 1)
df_test = df_test.drop(['Id'], axis = 1)
df = pd.concat([df_train, df_test])
df.head()              

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [68]:
df.shape

(2919, 79)

In [69]:
# 檢查 DataFrame 空缺值的狀態
def na_check(dataset):
    data_na = (dataset.isna().sum()/len(dataset))*100
    data_na = data_na.drop(data_na[data_na==0].index).sort_values(ascending = False)
    missing_data = pd.DataFrame({'Missing Ratio': data_na})
    display(missing_data(head()))

In [70]:
na_cheak(df)

,Missing Ratio
PoolQC,99.657417
MiscFeature,96.402878
Alley,93.216855
Fence,80.438506
FireplaceQu,48.646797
LotFrontage,16.649538
GarageFinish,5.447071
GarageYrBlt,5.447071
GarageQual,5.447071
GarageCond,5.447071


In [71]:
none_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'FireplaceQu', 'FireplaceQu', 'FireplaceQu', 
            'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
             'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'Functional', 'MSSubClass']
for col in none_cols:
    df[col] = df[col].fillna("None")
    
zero_cols = ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 
             'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea']
for col in zero_cols:
    df[col] = df[col].fillna(0)

In [72]:
df['MSZoning'].mode()[0]

'RL'

In [73]:
mode_cols = ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
for col in mode_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [74]:
df['LotFrontage'] = df.groupby("Neighborhood")['LotFrontage'].transform(lambda x : x.fillna(x.median()))
df = df.drop(['Utilities'], axis=1)

In [75]:
na_cheak(df)

,Missing Ratio


In [76]:
label_cols = ['MSSubClass', 'OverallCond', 'YrSold', 'MoSold']
for col in mode_cols:
    df[col] = df[col].astype(str)

In [87]:
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    LE = LabelEncoder()
    df[c] = LE.fit_transform(df[c])

In [89]:
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']   
print('Shape df: {}'.format(df.shape))
df.head()

Shape df: (2919, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalSF
0,5,RL,65.0,8450,1,1,3,Lvl,Inside,0,...,0,3,4,None,0,1,2,WD,Normal,2566.0
1,0,RL,80.0,9600,1,1,3,Lvl,FR2,0,...,0,3,4,None,0,4,1,WD,Normal,2524.0
2,5,RL,68.0,11250,1,1,0,Lvl,Inside,0,...,0,3,4,None,0,8,2,WD,Normal,2706.0
3,6,RL,60.0,9550,1,1,0,Lvl,Corner,0,...,0,3,4,None,0,1,0,WD,Abnorml,2473.0
4,5,RL,84.0,14260,1,1,0,Lvl,FR2,0,...,0,3,4,None,0,11,2,WD,Normal,3343.0


In [90]:
df = pd.get_dummies(df)
print(df.shape)

(2919, 221)


In [91]:
df.head()

,MSSubClass,LotFrontage,LotArea,Street,Alley,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,5,65.0,8450,1,1,3,0,7,4,2003,...,0,0,0,1,0,0,0,0,1,0
1,0,80.0,9600,1,1,3,0,6,7,1976,...,0,0,0,1,0,0,0,0,1,0
2,5,68.0,11250,1,1,0,0,7,4,2001,...,0,0,0,1,0,0,0,0,1,0
3,6,60.0,9550,1,1,0,0,7,4,1915,...,0,0,0,1,1,0,0,0,0,0
4,5,84.0,14260,1,1,0,0,8,4,2000,...,0,0,0,1,0,0,0,0,1,0


In [100]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

linear = LinearRegression()
GBR = GradientBoostingRegressor()
RFR = RandomForestRegressor()

In [105]:
lin_params = {'normalize': [True, False],
             'fit_intercept': [True, False]
             }
GBR_params = {'loss' : ['ls', 'lad', 'huber', 'quantile'],
              'learning_rate': [0.02, 0.03, 0.05, 0.9, 0.12, 0.15],
              'n_estimators': [10, 20 , 50, 100, 150],
              'tol' : [True, False]
             }
RFR_params = {'n_estimators': [10, 20 , 50, 100, 150],
              'max_depth': [2, 3, 4],
              'min_samples_split' : [2, 4, 7, 10],
              'max_features': [4, 6, 9]}

In [107]:
from sklearn.model_selection import RandomizedSearchCV

Lin_RS = RandomizedSearchCV(linear, lin_params, n_iter = 10, scoring = 'neg_mean_squared_error')
GBR_RS = RandomizedSearchCV(GBR, GBR_params, n_iter = 10, scoring = 'neg_mean_squared_error')
RFR_RS = RandomizedSearchCV(RFR, RFR_params, n_iter = 10, scoring = 'neg_mean_squared_error')

In [108]:
best_linear = Lin_RS.fit(train_X, train_Y)
best_GBR = GBR_RS.fit(train_X, train_Y)
best_RFR = RFR_RS.fit(train_X, train_Y)

C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Wong Kam Pui\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to sile

In [111]:
display(best_linear.best_estimator_.get_params())
display(best_GBR.best_estimator_.get_params())
display(best_RFR.best_estimator_.get_params())

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.12,
 'loss': 'lad',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 150,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'tol': True,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': 4,
 'max_features': 9,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [119]:
Lin = best_linear.best_estimator_
GBR = best_GBR.best_estimator_
RFR = best_RFR.best_estimator_

In [120]:
Lin.fit(train_X, train_Y)
GBR.fit(train_X, train_Y)
RFR.fit(train_X, train_Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
                      max_features=9, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [121]:
Lin_pred = Lin.predict(test_X)
GBR_pred = GBR.predict(test_X)
RFR_pred = RFR.predict(test_X)

In [127]:
blending_pred = Lin_pred*0.30 + GBR_pred * 0.67 + RFR_pred * 0.03

In [128]:
blending_pred

array([11.7202539 , 12.01964319, 12.15017487, ..., 12.10494738,
       11.69372244, 12.33040064])

In [129]:
np.expm1(blending_pred)

array([123037.66069764, 165982.42121583, 189126.16053278, ...,
       180762.96129227, 119816.18928136, 226476.44575313])